1. 문서의 내용을 읽는다
2. 문서를 쪼갠다
    - 토큰수 초과로 답변을 생성하지 못할 수 있고
    - 문서가 길면 (인풋이 길면) 답변 생성이 오래걸림   
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [3]:
#%pip install -qU  docx2txt 

In [4]:
# %pip install -qU langchain-text-splitters

In [5]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 200 # 겹치는 단위
)

loader = Docx2txtLoader('./tax_with_markdown.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [6]:
document_list[52]

Document(metadata={'source': './tax_with_markdown.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|--------------------------------------------|\n\n| 1,400만원 이하     | 과세표준의 6퍼센트                             |\n\n| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |\n\n| 5,000만원 초과   8,800만원 이하     | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하    | 3,706만원 + (8,800만원을 초과하는 금액의 35퍼센트)|\n\n| 1억5천만원 초과 3억원 이하         | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)|\n\n| 3억원 초과    5억원 이하         | 9,406만원 + (3억원을 초과하는 금액의 38퍼센트)   |\n\n| 5억원 초과      10억원 이하        | 1억 7,406만원 + (5억원을 초과하는 금액의 42퍼센트)|\n\n| 10억원 초과        | 3억 8,406만원 + (10억원을 초과하는 금액의 45퍼센트)|\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한

In [7]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model = 'text-embedding-3-large')

In [8]:
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = 'text-markdown2-index'
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key = pinecone_api_key)


database = PineconeVectorStore.from_documents(document_list,embedding,  index_name = index_name)

c:\Users\lw105\OneDrive\바탕 화면\LangChain\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'

In [10]:
retriever = database.as_retriever(search_kwargs={"k": 4})
retriever.invoke(query)

[Document(id='18aba434-8a9f-48cd-84bd-757f071b2f0d', metadata={'source': './tax_with_markdown.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|--------------------------------------------|\n\n| 1,400만원 이하     | 과세표준의 6퍼센트                             |\n\n| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |\n\n| 5,000만원 초과   8,800만원 이하     | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하    | 3,706만원 + (8,800만원을 초과하는 금액의 35퍼센트)|\n\n| 1억5천만원 초과 3억원 이하         | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)|\n\n| 3억원 초과    5억원 이하         | 9,406만원 + (3억원을 초과하는 금액의 38퍼센트)   |\n\n| 5억원 초과      10억원 이하        | 1억 7,406만원 + (5억원을 초과하는 금액의 42퍼센트)|\n\n| 10억원 초과        | 3억 8,406만원 + (10억원을 초과하는 금액의 45퍼센트)|\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소

In [11]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model = 'gpt-4o')


In [12]:
# %pip install -U langchain langchainhub --quiet

In [13]:
from langchain_classic import hub

prompt = hub.pull("rlm/rag-prompt")

In [14]:
from langchain_classic.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    chain_type_kwargs = {"prompt":prompt}
)

# query -> 직장인 -> 거주자 chain 추가
ai_message = qa_chain({"query": query})

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_21132\1502396560.py:10: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  ai_message = qa_chain({"query": query})


In [15]:
ai_message

{'query': '연봉 5천만원인 거주자의 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 거주자의 소득세는 624만원입니다. 5천만원은 종합소득 과세표준 5,000만원 이하에 해당되어 84만원 + (5,000만원 - 1,400만원) * 15%로 계산됩니다.'}

In [22]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

dictionary = ["사람을 나타내는 표현 -> 거주자"]

prompt = ChatPromptTemplate.from_template(f"""
    사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.     
    만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
    사전: {dictionary}  
    
    질문: {{question}}                              
""")

dictionary_chain = prompt | llm | StrOutputParser()

In [26]:
new_question = dictionary_chain.invoke({"question":query})

In [27]:
new_question

'질문: 연봉 5천만원인 거주자의 소득세는 얼마인가요?'

In [28]:
tax_chain = {"query":dictionary_chain}|qa_chain

In [29]:
ai_response = tax_chain.invoke({"question": query})

In [30]:
ai_response

{'query': '질문: 연봉 5천만원인 거주자의 소득세는 얼마인가요?',
 'result': '연봉 5천만 원인 거주자의 소득세는 624만 원입니다. 이는 종합소득 과세표준이 5천만 원 이하에 해당하며, 해당 구간에서 624만 원의 세율이 적용되기 때문입니다.'}